In [ ]:
import pandas as pd
import numpy as np
import os
from posixpath import join
from IPython.display import display
from IPython.display import Image

jpg_fld = "/mnt/datasets/mimic-cxr/mimic-cxr-jpg/physionet.org/files/mimic-cxr-jpg/2.0.0"
dcm_fld = "/mnt/datasets/mimic-cxr/mimic-cxr-dcm/physionet.org/files/mimic-cxr/2.0.0"

mimic has two versions:
- with dcm files
- with png files

metadata are split between the two versions. Specifically:
- record list, with path to image is in the dcm ver.
- metadata, with, for example, view position in the png ver.


In [ ]:
# dcm: read all files
ds = pd.read_csv( join(dcm_fld, "cxr-record-list.csv")).set_index(["subject_id", "study_id"])
print("record list - from dcm, shape:", ds.shape)
display(ds)

# jpg
metadata = pd.read_csv( join(jpg_fld, "mimic-cxr-2.0.0-metadata.csv") ).set_index(["subject_id", "study_id"])
print("metadata - from jpg, shape:", metadata.shape)
display(metadata)


# jpg


display some images

In [ ]:
vp = metadata.ViewPosition
print(vp.value_counts())
ds["ViewPosition"] = vp

display(ds)

# print("PA")
# pa = ds.loc[ds.ViewPosition == "PA", ["path"]]
# fns = pa.iloc[[0,1,2]].path
# for fn in fns:
#     fn = fn.replace(".dcm", ".jpg")
#     display(Image( join(jpg_fld, fn), width=224, height=224))

# print("AP")
# pa = ds.loc[ds.ViewPosition == "AP", ["path"]]
# fns = pa.iloc[[0,1,2]].path
# for fn in fns:
#     fn = fn.replace(".dcm", ".jpg")
#     display(Image( join(jpg_fld, fn), width=224, height=224))

# print("LATERAL")
# pa = ds.loc[ds.ViewPosition == "LATERAL", ["path"]]
# fns = pa.iloc[[0,1,2]].path
# for fn in fns:
#     fn = fn.replace(".dcm", ".jpg")
#     display(Image( join(jpg_fld, fn), width=224, height=224))

# print("XTABLE LATERAL")
# pa = ds.loc[ds.ViewPosition == "XTABLE LATERAL", ["path"]]
# print(len(pa))
# fns = pa.iloc[[0,1]].path
# for fn in fns:
#     fn = fn.replace(".dcm", ".jpg")
#     display(Image( join(jpg_fld, fn), width=224, height=224))

print("LL")
pa = ds.loc[ds.ViewPosition == "LL", ["path"]]
print(len(pa))
fns = pa.iloc[[10,20]].path
for fn in fns:
    fn = fn.replace(".dcm", ".jpg")
    display(Image( join(jpg_fld, fn), width=224, height=224))



- add column with labels
- add column with text

Read labels from chexpert/negbio in jpg folder
for each label
- +1 means positive mention
- -1 means negative mention
- 0 means uncertainty in the mention


In [ ]:
# jpg
chexpert = pd.read_csv( join(jpg_fld, "mimic-cxr-2.0.0-chexpert.csv"), na_filter=False).set_index(["subject_id", "study_id"])
print("chexpert - from jpg, shape:", chexpert.shape)
display(chexpert)

columns = chexpert.columns
print(f"n labels: {len(columns)}")

There are several strategies for managing the uncertain label (zero), as suggested by the authors themselves:
- 